In [3]:
pip install telethon

  Using cached pyaes-1.6.1-py3-none-any.whl
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 715.9/715.9 kB 925.4 kB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


### Получаем список каналов и записываем в текстовик, при необходимости редактируем

In [60]:
from telethon import TelegramClient
import asyncio
from telethon.tl.functions.messages import GetDialogsRequest
from telethon.tl.types import InputPeerEmpty
import pandas as pd
import json

#https://qna.habr.com/q/1276718 фикс бага
import nest_asyncio
nest_asyncio.apply()


# Данные API
with open('config.json', 'r') as file:
    config = json.load(file)

api_id = int(config['id'])
api_hash = config['hash']

# Создаем клиент
client = TelegramClient('test_tg', api_id, api_hash)



async def main():
    await client.start()

    # Получение списка чатов
    chats = []
    last_date = None
    size_chats = 200
    groups = []

    result = await client(GetDialogsRequest(
        offset_date=last_date,
        offset_id=0,
        offset_peer=InputPeerEmpty(),
        limit=size_chats,
        hash=0
    ))
    chats.extend(result.chats)

    # Фильтруем только каналы
    for chat in chats:
        try:
            if chat.broadcast:
                groups.append(chat.title)
        except:
            continue

    # Вывод списка каналов
    print('Выберите номер группы из перечня:')
    with open('channels.txt', 'w') as channels:
        for i, g in enumerate(groups):
            print(f"{i} - {g}")
            channels.write(g+'\n'*(g!=groups[-1]))



# Запускаем асинхронный код
async with client:
    client.loop.run_until_complete(main())


Выберите номер группы из перечня:
0 - Матч ТВ
1 - NN
2 - Антон Чехов. Лайфстайл
3 - Mash
4 - ТОПЛЕС
5 - Кинопоиск | Фильмы и сериалы
6 - Naked Science
7 - Неприятный канал
8 - Москвач • Новости Москвы
9 - Способы путешествoвать почти бесплатно (RU) | Vandrouki, Вандруки, вандроки — это мы
10 - Рестарт
11 - Старая Москва
12 - ПИК
13 - rutkismary
14 - ВкусВилл
15 - Авиакомпания «Победа»
16 - МУЗ-ТВ
17 - СЛЕЗЫ САТОШИ
18 - COZY HOME
19 - Альфа-Банк
20 - Доктор Комиссарова ☀️
21 - Алинамория
22 - twenty two store
23 - Адвокат туриста
24 - compozicia
25 - lag.gaet
26 - САТОШИ NEWS


In [55]:
from telethon import TelegramClient
import asyncio
from telethon.tl.functions.messages import GetDialogsRequest
from telethon.tl.types import InputPeerEmpty
import pandas as pd
from tqdm import tqdm

#https://qna.habr.com/q/1276718 фикс бага
import nest_asyncio
nest_asyncio.apply()


groups = []
with open('channels.txt') as channels:
    for line in channels.readlines():
        groups.append(line.replace('\n', ''))



async def main():
    await client.start()

    data = {}
    # Получаем сообщения из всех каналов
    for g in tqdm(groups):
        data[g]=[]
        messages = await client.get_messages(g, limit=1000)
        for message in messages:
            data[g].append(message.text)

    pd.DataFrame(data).to_csv('data.csv')



# Запускаем асинхронный код
async with client:
    client.loop.run_until_complete(main())


print(pd.read_csv('data.csv').drop(columns='Unnamed: 0').shape)

100%|███████████████████████████████████████████| 24/24 [02:39<00:00,  6.64s/it]


(1000, 24)


In [62]:
pd.read_csv('data.csv').drop(columns='Unnamed: 0')

,ТОПЛЕС,Кинопоиск | Фильмы и сериалы,Naked Science,Неприятный канал,Матч ТВ,Москвач • Новости Москвы,"Способы путешествoвать почти бесплатно (RU) | Vandrouki, Вандруки, вандроки — это мы",NN,Mash,Рестарт,...,ВкусВилл,Авиакомпания «Победа»,МУЗ-ТВ,СЛЕЗЫ САТОШИ,COZY HOME,Альфа-Банк,Доктор Комиссарова ☀️,Алинамория,Mash на спорте,САТОШИ NEWS
0,😍,NaN,"Ученые, которые совершили революцию в лечении ...",https://youtu.be/NF0vH2qXGLQ,**👑 Великий Император – в эфире Матч ТВ. Интер...,🪙 С 7 по 19 апреля пройдет акция «Монетная нед...,Без Азии никуда! Полёты из Иркутска и Владивос...,ИИ-iPhone без экрана: [появились](https://www....,"Искали молодёжный тариф с играми, музыкой и ре...",NaN,...,👀 **Рубрика ВкусВилл\n образовательный:*...,"**Мы присоединяемся ко всем тёплым словам, ска...",NaN,#ликвидации\n**За 24 ч ликвидировано позиций н...,Последний день дополнительной скидки на всю ко...,"**Разыгрываем 9 000 000 РУБЛЕЙ — среди всех, к...",Вы просили подборки: будут вам подборки\n\nНач...,**Боль неизбежна. Зачем принимать такую реальн...,NaN,#BTC #продажи\n**За последние 24 ч краткосрочн...
1,Я кончено в этом совсем не разбираюсь... Но ес...,NaN,В США успешно испытали генеративный чат-бот дл...,NaN,😳 Рамсдейл отыграл всего два матча в Чемпионши...,"⚡️ Владимир Путин подписал закон, запрещающий ...",Теперь точно пора брать: прямые рейсы из Сочи ...,Пополняем кошелек в Steam и покупаем игры в Pl...,ФНС заблокировала счета блогера Евгения Савина...,NaN,...,⛹️‍♂️ **Какой у вас дневной расход калорий?**\...,**ОТПУСК НА МАКСИМУМ!** \n\nБилеты в кармане. ...,NaN,"#starknet #проплатил #засадил\n**ПРО ТО, КАК М...",NaN,**Поздравляем Александра Овечкина с невероятны...,В Америке модный суперфуд сейчас это…кефир))\n...,**Что такое настоящие принятие**\n\n__Принимай...,NaN,#крипта #регулирование #ЭПР\n**Стали известны ...
2,Ручная работа всё таки,NaN,При сравнении десятков солнцеподобных звезд вы...,Папа в Вашингтоне дц,**☝️ Овечкин и Гретцки – абсолютно разные. Нел...,Ребята из Яндекс** ****🌍** Путешествий выдали ...,"Майские уже почти завтра, так что ловите подбо...",Новый умный гаджет от Xiaomi: [появился](https...,NaN,NaN,...,"Когда напомнили, что сегодня День супа, а мы —...","**Полетели Победой, а теперь добираются домой ...",NaN,NaN,NaN,NaN,Лето все громче напоминает о своем приближении...,"Сегодня дружеская рекомендация проекта, которы...",NaN,#Трамп #пошлины\n**Белый дом опроверг новость ...
3,Серьезно? Музыкальные часы 1810-х – это как-то...,NaN,"Воспалительные заболевания предложили лечить, ...","Очень рад, что ты признал вину. Это первый шаг...",**🙌 В семье Овечкина серьезно подходили к трен...,"⚡️ Путин подписал закон, который позволяет при...","Будет красиво: \n\nНеделя в ОАЭ, с вылетом из ...",Business Insider [опросил](https://www.busines...,Восстание машин в России началось с милого син...,NaN,...,ㅤ __⭐️занавес открывается⭐️__ㅤ ...,**Куда планируете лететь в апреле?** 🤔\n\n__Мы...,NaN,#мнение #сатоши #альтсезон\n**АЛЬТСЕЗОН И САТО...,NaN,NaN,"Да, мне смешно","**Главная ошибка, которая мешает пережить расс...",NaN,#Трамп #пошлины\n**Директор Национального экон...
4,"Просто чудо, вот это истинное проявление барок...",NaN,"Медики впервые описали, как кашель и больное с...",Едем в Вашингтон. Хорошего дня всем❤️,**🤼‍♂️ Сегодня стартует ЧЕ по борьбе. Увидим в...,⚠️ «Желтый» уровень опасности из-за гололедицы...,"Да-да, снова Пхукет: прямые рейсы из Красноярс...",Александр Овечкин побил рекорд Уэйна Гретцки п...,NaN,**Самая полная энциклопедия МЕМОВ **— псих из ...,...,"Какую? Расскажем чуть ниже, ведь церемония вот...",NaN,NaN,NaN,NaN,NaN,"Ставьте ❤️, если вы по голоду\n\nСтавьте 🔥, ес...",**Почему я не могу пережить расставание? \n**\...,NaN,"#ЕС #США #пошлины\n**В ЕС хотят достичь ""хорош..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,Праильна,NaN,**Самое популярное за неделю на Naked Science:...,"Прикиньте, у меня реально начали появляться мы...",🤔 Здесь был ~~Тимур~~ Георгий. \n\n📽 [FONBE